<a href="https://colab.research.google.com/github/ayoolafelix/DSN-BootCamp-Qualificaton/blob/master/DSN_Bootcamp_Qualification_2020_Final_Implement_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 
%matplotlib inline
 
import warnings
warnings.filterwarnings('ignore')

In [3]:
test = pd.read_csv('/content/gdrive/My Drive/DSN BootCamp Qualification/Test.csv')
train = pd.read_csv('/content/gdrive/My Drive/DSN BootCamp Qualification/Train.csv')
sample = pd.read_csv('/content/gdrive/My Drive/DSN BootCamp Qualification/SampleSubmission.csv')

In [4]:
dict_ = {'charge': 1,
         'lending': 0}

target_map = {
    'yes': 1,
    'no': 0}

train.default_status.replace(target_map, inplace = True)

train['form_field47'].replace(dict_, inplace = True)
test['form_field47'].replace(dict_, inplace = True)

In [5]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

In [6]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import hyperopt

In [7]:
features = train.select_dtypes(exclude = object).columns.drop(['default_status'])

In [8]:
# features

In [9]:
for col in features:
    train[col].fillna(-999, inplace = True)
    test[col].fillna(-999, inplace = True)

In [10]:
X = train[features]
y = train['default_status']

In [11]:
from sklearn.model_selection import StratifiedKFold

In [12]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0, 1])

In [17]:
# Specify number of folds
n_skf = 15
kf = StratifiedKFold(n_skf)


seed = 2020

## Manually tweak the hyperparameter, learning rate and n_estimators especially
params = {
    'n_estimators': 4000,
    'learning_rate': 0.0125,
    'objective': 'CrossEntropy',
      'eval_metric':'AUC',
    'random_seed': seed,
    'early_stopping_rounds': 15000,
    'use_best_model': True,
    'max_depth': 9,
    #'task_type' : 'GPU',
    #'model_size_reg': 0.8,
}

In [18]:
score_list = []
score = 0
test_oofs = []

for i, (tr_idx, vr_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]

    model = CatBoostClassifier(**params)
    model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=50)
    
    p = model.predict_proba(xval)[:, 1]
    sc = metric(yval, p)
    score_list.append(sc)
    score += sc/n_skf
    
    pred = model.predict_proba(test[features])[:, 1]
    test_oofs.append(pred)

    print('Fold {} : {}'.format(i, sc))

print()
print()
print('Avg log : ', score)

0:	test: 0.7955491	best: 0.7955491 (0)	total: 100ms	remaining: 6m 41s
50:	test: 0.8173063	best: 0.8173063 (50)	total: 4.87s	remaining: 6m 17s
100:	test: 0.8199637	best: 0.8199637 (100)	total: 9.48s	remaining: 6m 6s
150:	test: 0.8220731	best: 0.8220731 (150)	total: 14s	remaining: 5m 58s
200:	test: 0.8232955	best: 0.8232955 (200)	total: 18.7s	remaining: 5m 54s
250:	test: 0.8241139	best: 0.8241139 (250)	total: 23.3s	remaining: 5m 47s
300:	test: 0.8251762	best: 0.8251762 (300)	total: 27.8s	remaining: 5m 41s
350:	test: 0.8257775	best: 0.8257775 (350)	total: 32.3s	remaining: 5m 35s
400:	test: 0.8263629	best: 0.8263776 (399)	total: 36.9s	remaining: 5m 30s
450:	test: 0.8268525	best: 0.8268525 (450)	total: 41.3s	remaining: 5m 25s
500:	test: 0.8273406	best: 0.8273883 (497)	total: 45.8s	remaining: 5m 20s
550:	test: 0.8278656	best: 0.8278780 (548)	total: 50.5s	remaining: 5m 15s
600:	test: 0.8279047	best: 0.8279505 (575)	total: 55s	remaining: 5m 11s
650:	test: 0.8280757	best: 0.8280780 (644)	total:

In [19]:
 f"{n_skf} fold CV, score: {score}" #Increase in local cv score should probably lead to increase in Leaderboard score

'15 fold CV, score: 0.8419600870684594'

### Since we are using cross validation, We'll have `n` different predictions (where n is the number of fold)

In [20]:
oof_prediction = pd.DataFrame(test_oofs).T

In [21]:
oof_prediction.columns = ['fold_'+ str(i) for i in range(1, n_skf + 1)]

In [22]:
oof_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10,fold_11,fold_12,fold_13,fold_14,fold_15
0,0.344007,0.254141,0.301838,0.308516,0.292236,0.271980,0.309695,0.314646,0.305176,0.280468,0.295101,0.298850,0.290020,0.316961,0.308404
1,0.402076,0.366217,0.397219,0.416626,0.463205,0.438064,0.380971,0.446061,0.454531,0.418834,0.339310,0.402898,0.447511,0.432525,0.401098
2,0.373173,0.375635,0.350215,0.366206,0.390525,0.384102,0.370623,0.379366,0.379628,0.404294,0.385584,0.381042,0.344480,0.340050,0.355925
3,0.739581,0.737789,0.753064,0.772047,0.766281,0.771351,0.741851,0.760718,0.752846,0.759183,0.792404,0.768607,0.752516,0.752769,0.758265
4,0.119907,0.120978,0.176038,0.179754,0.103203,0.133148,0.144337,0.155651,0.143312,0.132412,0.143476,0.109692,0.168806,0.152760,0.154055


In [23]:
### We take the mean of all 5 predictions to generate submission

In [24]:
sample['default_status'] = np.mean(test_oofs, axis = 0) 

In [25]:
sample.to_csv('submit.csv', index = False)

### Suggestions:
- Base your feature selection on the local cv score i.e When you add, remove or engineer feature check `score` (cell 17)
- Increase number of fold to 10, 15. See how score imporve on leaderboard.
- Try differnt value of learning rate. I'll advice anything < 0.01
- Create as many features as possible.
- Find code for Xgboost and lgbm using this method of modelling, or better still tweak this one (mind you, names of parameters may be slightly different.

In [ ]:
#sample['default_status'] = oof_prediction['fold_13']
#sample.to_csv('submit.csv', index = False)